In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from matplotlib import rcParams

rcParams["savefig.dpi"] = 200
rcParams["font.size"] = 8

import warnings

warnings.filterwarnings("ignore")

# Create your own region

% comment: leave next cell as RawNBConvert -> else the warning mesage is not correctly displayed

Creating own regions is straightforward. Import regionmask and check the version:

In [ ]:
import regionmask

regionmask.__version__

Import numpy

In [ ]:
import numpy as np

Assume you have two custom regions in the US, you can easily use these to create `Regions`:

In [ ]:
US1 = np.array([[-100.0, 30], [-100, 40], [-120, 35]])
US2 = np.array([[-100.0, 30], [-80, 30], [-80, 40], [-100, 40]])

regionmask.Regions([US1, US2])

If you want to set the `names` and `abbrevs` yourself you can still do that:

In [ ]:
names = ["US_west", "US_east"]
abbrevs = ["USw", "USe"]

USregions = regionmask.Regions([US1, US2], names=names, abbrevs=abbrevs, name="US")
USregions

Again we can plot the outline of the defined regions

In [ ]:
ax = USregions.plot(label="abbrev")

# load cartopy
import cartopy.crs as ccrs

# fine tune the extent
ax.set_extent([225, 300, 25, 45], crs=ccrs.PlateCarree())

and obtain a mask:

In [ ]:
import numpy as np

# define lat/ lon grid
lon = np.arange(200.5, 330, 1)
lat = np.arange(74.5, 15, -1)

# for the plotting
lon_edges = np.arange(200, 331, 1)
lat_edges = np.arange(75, 14, -1)

mask = USregions.mask(lon, lat)

In [ ]:
import matplotlib.pyplot as plt

ax = plt.subplot(111, projection=ccrs.PlateCarree())

# pcolormesh does not handle NaNs, requires masked array
mask_ma = np.ma.masked_invalid(mask)

h = ax.pcolormesh(
    lon_edges,
    lat_edges,
    mask_ma,
    transform=ccrs.PlateCarree(),
    cmap="viridis",
)

ax.coastlines()

# add the outlines of the regions
USregions.plot_regions(ax=ax, add_label=False)

ax.set_extent([225, 300, 25, 45], crs=ccrs.PlateCarree())

## Use shapely Polygon

If you have the region defined as a shapely polygon, this also works:

In [ ]:
from shapely.geometry import Polygon, MultiPolygon

US1_poly = Polygon(US1)
US2_poly = Polygon(US2)

US1_poly, US2_poly

In [ ]:
USregions_poly = regionmask.Regions([US1_poly, US2_poly])

USregions_poly

## Create Regions with MultiPolygon and interiors

Create two discontiguous regions and combine them to one. Add a hole to one of the regions

In [ ]:
US1_shifted = US1 - (5, 0)
US2_hole = np.array([[-98.0, 33], [-92, 33], [-92, 37], [-98, 37], [-98.0, 33]])

Create `Polygons`, a `MultiPolygon`, and finally `Regions`

In [ ]:
US1_poly = Polygon(US1_shifted)
US2_poly = Polygon(US2, holes=[US2_hole])

US_multipoly = MultiPolygon([US1_poly, US2_poly])

USregions_poly = regionmask.Regions([US_multipoly])

In [ ]:
USregions_poly.plot();

Create a mask:

In [ ]:
mask = USregions_poly.mask(lon, lat)

and plot it:

In [ ]:
ax = plt.subplot(111, projection=ccrs.PlateCarree())

mask.plot(transform=ccrs.PlateCarree(), add_colorbar=False)

ax.coastlines();